<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Generate/ProtXLNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Generate protein sequences using ProtXLNet pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 1.1MB 8.4MB/s 
     |████████████████████████████████| 890kB 31.7MB/s 
     |████████████████████████████████| 1.1MB 42.8MB/s 
     |████████████████████████████████| 3.0MB 50.1MB/s 


In [2]:
import torch
from transformers import XLNetLMHeadModel, XLNetTokenizer,pipeline
import re
import os
import requests
from tqdm.auto import tqdm

<b>2. Load the vocabulary and ProtXLNet Model<b>

In [3]:
tokenizer = XLNetTokenizer.from_pretrained("Rostlab/prot_xlnet", do_lower_case=False)

In [4]:
model = XLNetLMHeadModel.from_pretrained("Rostlab/prot_xlnet")

/usr/local/lib/python3.6/dist-packages/transformers/configuration_xlnet.py:212: FutureWarning: This config doesn't use attention memories, a core feature of XLNet. Consider setting `men_len` to a non-zero value, for example `xlnet = XLNetLMHeadModel.from_pretrained('xlnet-base-cased'', mem_len=1024)`, for accurate training performance as well as an order of magnitude faster inference. Starting from version 3.5.0, the default parameter will be 1024, following the implementation in https://arxiv.org/abs/1906.08237
  FutureWarning,


<b>3. Load the model into the GPU if avilabile and switch to inference mode<b>

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
model = model.to(device)
model = model.eval()

<b>4. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (unk)<b>

In [7]:
sequences_Example = "A E T C Z A O"

In [8]:
sequences_Example = re.sub(r"[UZOB]", "<unk>", sequences_Example) 

<b>5. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [9]:
ids = tokenizer.encode(sequences_Example, add_special_tokens=False)

In [10]:
input_ids = torch.tensor(ids).unsqueeze(0).to(device)

<b>6. Generate Protein Sequence<b>

In [11]:
max_length = 100
temperature = 1.0
k = 0
p = 0.9
repetition_penalty = 1.0
num_return_sequences = 3

In [12]:
output_ids = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=k,
        top_p=p,
        repetition_penalty=repetition_penalty,
        do_sample=True,
        num_return_sequences=num_return_sequences,
    )

In [13]:
output_sequences = [" ".join(" ".join(tokenizer.decode(output_id)).split()) for output_id in output_ids]

In [14]:
print('Generated Sequences\n')
for output_sequence in output_sequences:
  print(output_sequence)

Generated Sequences

A E T C X A X A A E T C S A A A A E T C S A A A A E T C S A A A A E T C S A A A A E T C S A A A A E T C S A A A A E T C S A A A A E T C S A A A A E T C S A A A A E T C S A A A A E T C S A A A A E T C
A E T C X A X A A E T C M A M A A E T C M A M A A E T C M A M A A E T C M A M A A E T C M A M A A E T C M A M A A E T C M A M A A E T C M A M A A E T C M A M A A E T C M A M A A E T C M A M A A E T C
A E T C X A X A A E T C S A K A A E T C S A K A A E T C S A K A A E T C S A K A A E T C S A K A A E T C S A K A A E T C S A K A A E T C S A K A A E T C S A K A A E T C S A K A A E T C S A K A A E T C
